In [17]:
import requests
import pandas as pd
import csv
import urllib.request
from bs4 import BeautifulSoup

Print version numbers for reproducibility

In [20]:
%load_ext watermark
%watermark
%watermark --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-08-06T20:32:48.524504+02:00

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.25.0

Compiler    : Clang 15.0.7 
OS          : Darwin
Release     : 22.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit

requests: 2.32.2
bs4     : 4.12.3
csv     : 1.0
pandas  : 1.5.3



In [2]:
# rename the folder path before running
project_folder = 'C:\\Users\\tkhishigsure\\OneDrive - The University of Melbourne\\Documents\\github\\lexical_elaboration'

In [3]:
# load words of interest
focus_list = set()
with open(project_folder+'\\data\\biladataset\\bila_long_nounverbadj_unfiltered_full.csv', 'r', encoding='utf-8') as f:
    for line in f:
      parts = line.rstrip("\n").split(",")
      focus_list.add(parts[1])

In [4]:
def printResults(word, language, meaning, provenance):
  f = open(project_folder+'\\data\\forpreprocessing\\wiktionary_forms.tsv','a', encoding='utf-8')
  f.write(word+"\t"+language+"\t"+str(meaning)+"\t"+provenance+"\n")
  f.close()

In [5]:
# print headers
printResults("word", "language_name", "same_meaning", "provenance")

In [6]:
import re

def extract_meanings(description_line):
    pattern = r'{{l\|en\|(.*?)}}'
    copyline= ''+ description_line
    
    # Use re.findall to find all matches of the pattern in the input string
    matches = re.findall(pattern, description_line, re.DOTALL)  # re.DOTALL allows matching across multiple lines
    
    if len(matches) == 0:
        #English|'+word+']]
        pattern = r'#English\|(.*?)\]\]'
        matches = re.findall(pattern, copyline)
        
    if len(matches) == 0:
        pattern = r'\[\[(.*?)\]\]'

        # Use re.findall to find all matches of the pattern in the input string
        matches = re.findall(pattern, copyline)
    
    ret = []

    for match in matches:
        if '|' in match:
            ret.append(match.split('|')[0])
        else:
            ret.append(match)

    # Return the list of matches
    return ret

# Example usage
html_string = "# {{l|en|local guard2|suf}} {{l|en|guard}}"
html_string = "# [[local guard|English]] [[guard]]"
#html_string = "# [[#English|some word guard]] [[guard]]"
paragraphs = extract_meanings(html_string)
#print(paragraphs)

In [7]:
import re

#helper functions
def preprocessingLine(line):
    line = line.strip()
    return line


def extractTitle(line):
    #print(line.index('</title>'))
    return line[7:line.index('</title>')]

def extract_meanings(description_line):
    pattern = r'{{l\|en\|(.*?)}}'
    copyline= ''+ description_line
    
    # Use re.findall to find all matches of the pattern in the input string
    matches = re.findall(pattern, description_line, re.DOTALL)  # re.DOTALL allows matching across multiple lines
    
    if len(matches) == 0:
        #English|'+word+']]
        pattern = r'#English\|(.*?)\]\]'
        matches = re.findall(pattern, copyline)
        
    if len(matches) == 0:
        pattern = r'\[\[(.*?)\]\]'

        # Use re.findall to find all matches of the pattern in the input string
        matches = re.findall(pattern, copyline)
    
    ret = []

    for match in matches:
        if '|' in match:
            ret.append(match.split('|')[0])
        else:
            ret.append(match)

    # Return the list of matches
    return ret

def extract_forms1(description_line):
    pattern = r'-verb form of\|(.*?)}}'
    copyline= ''+ description_line
    
    # Use re.findall to find all matches of the pattern in the input string
    matches = re.findall(pattern, description_line, re.DOTALL)  # re.DOTALL allows matching across multiple lines
    
    ret = []

    for match in matches:
        if '|' in match:
            ret.append(match.split('|')[0])
        else:
            ret.append(match)

    # Return the list of matches
    return ret

def extract_forms2(description_line, head):
    pattern = r''+head+r'\|(.*?)}}'
    copyline= ''+ description_line
    
    # Use re.findall to find all matches of the pattern in the input string
    matches = re.findall(pattern, description_line, re.DOTALL)  # re.DOTALL allows matching across multiple lines
    
    ret = []

    for match in matches:
        if '|' in match:
            ret.append(match.split('|')[1])
        else:
            ret.append(match)

    # Return the list of matches
    return ret

html_string = "# {{gl-verb form of|danzar}}"

paragraphs = extract_forms1(html_string)
#print(paragraphs)

html_string = "# {{plural of|it|clientela}}"

paragraphs = extract_forms2(html_string,"plural of|"[:-1])
#print(paragraphs)

def identifyLine(line):
    if '<title>' in line:
        return 1
    
    # Language name
    if len(line) > 4 and line.startswith('==') and line[3]!='=' and line.endswith('==') and line[-3]!='=':
        return 2
    
    # gloss line for word
    if line.startswith('# '):
        return 3
    
    # others
    return 0

In [8]:
lan_maps = {}
with open(project_folder+'\\rawdata\\downloaded\\wiktionary_langs.tsv', 'r', encoding='utf-8') as f_in:
    for line in f_in:
        elements = line.strip().split('\t')
        lan_maps[elements[1]] = elements[0]

#print(lan_maps['French'])

In [9]:
cognates_with_lan = set()
cognates = set()
with open(project_folder+'\\rawdata\\downloaded\\CogNet-v2.0.tsv', 'r', encoding='utf-8') as f_in:
    for line in f_in:
        elements = line.strip().split('\t')
        if elements[1] == 'eng':
            cognates_with_lan.add(elements[2]+'_'+elements[3]+'_'+elements[4])
            cognates.add(elements[2]+'_'+elements[4])
        if elements[3] == 'eng':
            cognates_with_lan.add(elements[4]+'_'+elements[1]+'_'+elements[2])
            cognates.add(elements[4]+'_'+elements[2])

#print(len(cognates_with_lan),len(cognates))

In [10]:
from collections import defaultdict

def wiktionarySuffixParse(input_file):
    ret = defaultdict(list)
    it = 0
    prev_line = ''
    enabled = False
    meaning = False
    language='n.a'
    #print('Suffix scrap beginning...')
    with open(input_file, 'r', encoding='utf-8') as f_in:
        for line in f_in:
            it+=1
            c_line = preprocessingLine(line)
            cat = identifyLine(c_line)
            if cat == 1:
                word = extractTitle(c_line)
                if word.startswith('-') is False:
                    enabled = False
                    continue                
                if word[1:] in focus_list:
                    enabled = True
                else:
                    enabled = False
                language = 'n.a'
            if enabled:
                # other line
                if cat == 0:
                    prev_line = c_line
                    continue
                # word title as new page and word
                elif cat == 2: # new language line
                    #extracting language as ==French==
                    language = c_line[2:len(c_line)-2]
                    if language!='English':
                        ret[word[1:]].append(language)
            #if it % 20000000 == 0:
                #print(it)
            #prev_line = c_line
    #print('Suffix scrap ending...')
    ans = {}
    for k, v in ret.items():
        for lan in v:
            ans[(k,lan)] = 0
    return ans

In [11]:
heads = ['plural of|','inflection of|','infl of|','pl of|','verb form of|']
def wiktionaryParse(input_file):
    it = 0
    prev_line = ''
    enabled = False
    meaning = False
    language='n.a'
    provenance = 'n.a'
    answers = []
    
    word_lan_set = wiktionarySuffixParse(input_file)
    #print('Beginning...')
    with open(input_file, 'r', encoding='utf-8') as f_in:
        for line in f_in:
            it+=1
            c_line = preprocessingLine(line)
            cat = identifyLine(c_line)
            if cat == 1:
                if language!='n.a':
                    if (word, language) in word_lan_set:
                        meaning = False
                        provenance = 'suffix'
                    word_lan_set[(word, language)] = 1
                    answers.append((word, language, meaning, provenance))
                    #printResults(word, language, meaning)
                word = extractTitle(c_line)
                if word in focus_list:
                    enabled = True
                else:
                    enabled = False
                language = 'n.a'
            if enabled:
                # other line
                if cat == 0:
                    prev_line = c_line
                    continue
                # word title as new page and word
                elif cat == 2: # new language line
                    if language!='n.a':
                        if (word, language) in word_lan_set:
                            meaning = False
                            provenance = 'suffix'
                        word_lan_set[(word, language)] = 1
                        answers.append((word, language, meaning,provenance))
                        
                        #printResults(word, language, meaning)
                    #extracting language as ==French==
                    language = c_line[2:len(c_line)-2]
                    meaning = False
                    if language == 'English':
                        meaning = True
                        provenance = 'Wiktionary'
                    elif language in lan_maps:
                        search = word+'_'+lan_maps[language]+'_'+word
                        if search in cognates_with_lan:
                            meaning = True
                            provenance = 'CogNet'
                elif cat == 3: # new entry line
                    meanings = extract_meanings(c_line)
                    for m in meanings:
                        if word == m or word +' ' in m or ' '+word in m or word +'-' in m or '-'+word in m:
                            meaning = True
                            provenance='Wiktionary'
                    if len(meanings) == 0:
                        forms = []
                        if '-verb form of|' in c_line:
                            forms = extract_forms1(c_line)
                        else:
                            for head in heads:
                                if head in c_line:
                                    forms = extract_forms2(c_line, head[:-1])
                                    break
                        for form in forms:
                            form = form.strip()
                            search = word+'_'+form
                            if search in cognates:
                                meaning = True
                                provenance = 'CogNet'
                                break
            #if it % 1000000 == 0:
                #print(it)
            #prev_line = c_line
    for k, v in word_lan_set.items():
        if v == 0:
            answers.append((k[0],k[1],False, 'suffix'))
    sorted_list = sorted(answers)
    for word, language, meaning, provenance in sorted_list:
        printResults(word, language, meaning, provenance)
    return

In [12]:
resource_name = 'enwiktionary-20240320'
wiktionaryParse(project_folder+'\\rawdata\\downloaded\\'+resource_name+'-pages-articles-multistream.xml')